# Generate Tracing Framework

This is another script, this time for creating the necessary information for gathering
tracing information on a per-call basis.

* It reads fuse_lowlevel.h and finds the code definition(s)
* It generates a symbolic name for the entry point
* It generates a header file and records the entry points

It sure would be great if it also added it to the file(s)...



In [1]:
import os
import sys


In [2]:
fuse_lowlevel_h = "../../include/fuse_lowlevel.h"

with open(fuse_lowlevel_h, "rt") as fd:
    fuse_ll_lines = fd.readlines()

index = 0
functions = {}
for l in fuse_ll_lines:
    # Step (1): find the start of the operations definition
    index = index + 1
    if 'struct fuse_lowlevel_ops' in l: break
while index < len(fuse_ll_lines):
    if 'void (*' in fuse_ll_lines[index]:
        func = fuse_ll_lines[index][2+fuse_ll_lines[index].find('('):fuse_ll_lines[index].find(')')]
        functions[func] = fuse_ll_lines[index].rstrip()
        while True:
            if ');' in fuse_ll_lines[index]: break
            index = index + 1
            functions[func] += fuse_ll_lines[index].rstrip()
    if fuse_ll_lines[index].startswith('};'): break
    index = index + 1
for func in functions:
    # functions[func] = functions[func].replace('\t', '')
    functions[func] = ' '.join(functions[func].split())
for func in functions:
    print(functions[func])

void (*init)(void *userdata, struct fuse_conn_info *conn);
void (*destroy)(void *userdata);
void (*lookup)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*forget)(fuse_req_t req, fuse_ino_t ino, uint64_t nlookup);
void (*getattr)(fuse_req_t req, fuse_ino_t ino, struct fuse_file_info *fi);
void (*setattr)(fuse_req_t req, fuse_ino_t ino, struct stat *attr, int to_set, struct fuse_file_info *fi);
void (*readlink)(fuse_req_t req, fuse_ino_t ino);
void (*mknod)(fuse_req_t req, fuse_ino_t parent, const char *name, mode_t mode, dev_t rdev);
void (*mkdir)(fuse_req_t req, fuse_ino_t parent, const char *name, mode_t mode);
void (*unlink)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*rmdir)(fuse_req_t req, fuse_ino_t parent, const char *name);
void (*symlink)(fuse_req_t req, const char *link, fuse_ino_t parent, const char *name);
void (*rename)(fuse_req_t req, fuse_ino_t parent, const char *name, fuse_ino_t newparent, const char *newname, unsigned int flags);
void (*li

In [51]:
symbols = []
calls = []
for func in functions:
    cfunc = functions[func]
    proto_start = cfunc.find('(*')
    proto_end = cfunc.find(')(')
    proto_end_length = 1
    if proto_end <= proto_start:
        proto_end = cfunc.find(') (')
        proto_end_length = 2
    assert proto_end > proto_start, '{}: start = {}, end = {}'.format(func, proto_start, proto_end)
    statement_end = cfunc.find(';')
    sym = 'BITBUCKET_CALL_' + cfunc[proto_start + 2: proto_end]
    symbols.append(sym.upper())
    calls.append(cfunc[proto_start + 2: proto_end])

# At this point we have prospective symbols and calls
base = 100
index = 0
defs = [
    '//',
    '// (C) Copyright 2020 Tony Mason',
    '// All Right Reserved',
    '//',
    '',
    '',
    '#if !defined(__BITBUCKET_CALLS_H__)',
    '#define __BITBUCKET_CALLS_H__ (1)',
    '#define BITBUCKET_CALL_BASE ({})'.format(base),
    '#include <time.h>',
    '#include <stdint.h>',
    '#include <assert.h>',
    '',
    '',
]

timespec = '''

static inline void timespec_diff(struct timespec *begin, struct timespec *end, struct timespec *diff) {
    struct timespec result = {.tv_sec = 0, .tv_nsec = 0};
    assert((end->tv_sec > begin->tv_sec) || ((end->tv_sec == begin->tv_sec) && end->tv_nsec >= begin->tv_nsec));
    result.tv_sec = end->tv_sec - begin->tv_sec;
    if (end->tv_nsec < begin->tv_nsec) {
        result.tv_sec--;
        result.tv_nsec = (long)1000000000 + end->tv_nsec - begin->tv_nsec;
    }
    *diff = result;
}

static inline void timespec_add(struct timespec *one, struct timespec *two, struct timespec *result) {
    result->tv_sec = one->tv_sec + two->tv_sec;
    result->tv_nsec = one->tv_nsec + two->tv_nsec;
    while ((long)1000000000 >= result->tv_nsec) {
        result->tv_sec++;
        result->tv_nsec -= (long)1000000000;
    }
}

'''
for sym in symbols:
    index = index + 1
    sym = '#define ' + sym + ' (BITBUCKET_CALL_BASE + {})'.format(index)
    defs.append(sym)
defs.append('#define BITBUCKET_CALLS_MAX ({})'.format(base + index))
defs.append('')
defs.append('')
defs.append('')
defs.append('typedef struct _bitbucket_call_statistics {')
defs.append('\tuint64_t        Calls;')
defs.append('\tuint64_t        Success;')
defs.append('\tuint64_t        Failure;')
defs.append('\tstruct timespec ElapsedTime;')
defs.append('} bitbucket_call_statistics_t;\n')
defs.append('extern bitbucket_call_statistics_t BitbucketCallStatistics[{}];'.format(index))
defs.append('\n')
defs.append('')
defs.append(timespec)
defs.append('')
defs.append('static inline void bitbucket_count_call(uint8_t Call, uint8_t Success, struct timespec *Elapsed) {')
defs.append('\tassert((Call > BITBUCKET_CALL_BASE) && (Call < BITBUCKET_CALLS_MAX));')
defs.append('\tassert((0 == Success) || (1 == Success));')
defs.append('\tBitbucketCallStatistics[Call - BITBUCKET_CALL_BASE].Calls++;')
defs.append('\tBitbucketCallStatistics[Call - BITBUCKET_CALL_BASE].Success += Success;')
defs.append('\tBitbucketCallStatistics[Call - BITBUCKET_CALL_BASE].Failure += !Success;')
defs.append('\ttimespec_add(&BitbucketCallStatistics[Call - BITBUCKET_CALL_BASE].ElapsedTime, Elapsed, &BitbucketCallStatistics[Call - BITBUCKET_CALL_BASE].ElapsedTime);')
defs.append('}')
defs.append('')
defs.append('#endif // ____BITBUCKET_CALLS_H__')

with open('bitbucketcalls.h', 'wt') as fd:
    fd.write('\n'.join(defs))

In [52]:
files = []
for func in functions:
    fname = '{}.c'.format(func)
    if os.path.exists(fname): files.append(func)
print(files)

['init', 'lookup', 'forget', 'getattr', 'setattr', 'readlink', 'mknod', 'mkdir', 'unlink', 'rmdir', 'symlink', 'rename', 'link', 'open', 'read', 'write', 'flush', 'release', 'fsync', 'opendir', 'readdir', 'releasedir', 'fsyncdir', 'statfs', 'setxattr', 'getxattr', 'listxattr', 'removexattr', 'access', 'create', 'getlk', 'setlk', 'bmap', 'ioctl', 'poll', 'write_buf', 'retrieve_reply', 'forget_multi', 'flock', 'fallocate', 'readdirplus', 'copy_file_range', 'lseek']


In [57]:
def generate_wrapper(func, line):
    line = line.strip()
    params = []
    args = line[line.find('(') + 1:line.find(')')].split(',')
    for arg in args:
        arg = arg.split(' ')[-1]
        if '*' == arg[0]: arg = arg[1:]
        params.append(arg)
    params = ', '.join(params)
    wrapper = [
        'static int bitbucket_internal_{}'.format(func) + line[line.find('('):] + ';',
        '\n',
        line,
        '{',
        '\tstruct timespec start, stop, elapsed;',
        '\tint status, tstatus;',
        '',
        '\ttstatus = clock_gettime(CLOCK_MONOTONIC_RAW, &start);',
        '\tassert(0 == tstatus);',
        '\tstatus = bitbucket_internal_{}'.format(func) + '({});'.format(params),
        '\ttstatus = clock_gettime(CLOCK_MONOTONIC_RAW, &stop);',
        '\tassert(0 == tstatus);',
        '\ttimespec_diff(&start, &stop, &elapsed);',
        '\tbitbucket_count_call(BITBUCKET_CALL_{}, status ? 0 : 1, &elapsed);'.format(func.upper()),
        '}\n',
        ''
    ]
    line = line.replace('void', 'int', 1)
    return (wrapper, line.replace('bitbucket_{}'.format(func), 'bitbucket_internal_{}'.format(func)))

In [58]:
# Now, I'm looking for the bitbucket_xxx function inside these files.

for f in files:
    with open('{}.c'.format(f), 'rt') as fd:
        with open('{}_new.c'.format(f), 'wt') as nfd:
            for line in fd.readlines():
                if 'bitbucket_{}'.format(f) in line: 
                    wrapper, line = generate_wrapper(f, line)
                    nfd.write('\n'.join(wrapper))
                    line = 'static ' + line + '\n'
                nfd.write(line)
                if '#include "bitbucket.h"' in line:
                    nfd.write('#include "bitbucketcalls.h"\n')
    os.rename('{}_new.c'.format(f), '{}.c'.format(f))
    print(f)

init
lookup
forget
getattr
setattr
readlink
mknod
mkdir
unlink
rmdir
symlink
rename
link
open
read
write
flush
release
fsync
opendir
readdir
releasedir
fsyncdir
statfs
setxattr
getxattr
listxattr
removexattr
access
create
getlk
setlk
bmap
ioctl
poll
write_buf
retrieve_reply
forget_multi
flock
fallocate
readdirplus
copy_file_range
lseek
